In [1]:
import os

from census import Census
import pandas as pd

### Setup the Census API Key and Year

In [2]:
c = Census('29299f5c66e3e1fb3f945df3aa82db684570cd3b', year=2019)

state_county_fips = [
    ('47', '065'),  # Hamilton County, TN
    ('13', '047'),  # Catoosa County, GA
    ('13', '083'),  # Dade County, GA
    ('13', '295')   # Walker County, GA
]

This notebook relies on the [Census Python package](https://github.com/datamade/census) to access the [U.S. Census API](https://api.census.gov/data/2019/acs/acs5/groups.html). The tables used in this Notebook include:

- [B11005: HOUSEHOLDS BY PRESENCE OF PEOPLE UNDER 18 YEARS BY HOUSEHOLD TYPE](https://api.census.gov/data/2019/acs/acs5/groups/B11005.html)
- [B08201: HOUSEHOLD SIZE BY VEHICLES AVAILABLE](https://api.census.gov/data/2019/acs/acs5/groups/B08201.html)
- [B26001: GROUP QUARTERS POPULATION](https://api.census.gov/data/2019/acs/acs5/groups/B26001.html)
- [B25010: AVERAGE HOUSEHOLD SIZE OF OCCUPIED HOUSING UNITS BY TENURE](https://api.census.gov/data/2019/acs/acs5/groups/B25010.html)

### Read in the Existing Tract Data for Reference

In [3]:
tract_data = pd.read_excel(r'C:\Users\USYS671257\WSP O365\chattanooga-modeling - RTP TDM Update\Deliverables\PopSyn Inputs.xlsx',
              sheet_name='tractdata')
tract_data['GEOID10'] = tract_data['GEOID10'].apply(lambda x: '{:.0f}'.format(x))

### A Quick Consistency Test

In [4]:
hh_size = tract_data[['HH_1', 'HH_2', 'HH_3', 'HH_4PLUS']].sum(axis=1)
hh_work = tract_data[['HH_NO_WORK', 'HH_1_WORK', 'HH_2_WORK', 'HH_3PLUS_WORK']].sum(axis=1)
pd.testing.assert_series_equal(hh_size, hh_work)

### Get Households with and without Children

In [5]:
hh = []
hhc = []
for state, co in state_county_fips:
    # Get the Total Count of Households
    hh.append(pd.DataFrame(c.acs5.get(('NAME', 'B11005_001E'),
                            {
                                'for': 'tract:*',
                                'in': 'state:{} county:{}'.format(state, co)
                            }
                )))

    # Get the count of households with children
    hhc.append(pd.DataFrame(c.acs5.get(('NAME', 'B11005_002E'),
              {
                  'for': 'tract:*',
                  'in': 'state:{} county:{}'.format(state, co)
                            }
                )))

hh = pd.concat(hh)
hhc = pd.concat(hhc)

hh = pd.merge(hh, hhc[['state', 'county', 'tract', 'B11005_002E']], on=['state', 'county', 'tract'], how='outer')

# Percent of HH with Children
hh['hhc'] = hh['B11005_002E'] / hh['B11005_001E']

# GEOID for merge
hh['GEOID10'] = hh.apply(lambda x: '{}{}{}'.format(x.state, x.county, x.tract), axis=1)

hh = pd.merge(tract_data, hh[['GEOID10', 'hhc']], on='GEOID10')

hh['HH'] = hh[['HH_1', 'HH_2', 'HH_3', 'HH_4PLUS']].sum(axis=1)
hh['HH_CHILD_NEW'] = (hh['HH'] * hh['hhc']).round()
hh['HH_NO_CHILD_NEW'] = hh['HH'] - tract_data['HH_CHILD_NEW']
hh[['GEOID10', 'HH_CHILD_NEW', 'HH_NO_CHILD_NEW']]#.to_clipboard(sep='\t')

,GEOID10,HH_CHILD_NEW,HH_NO_CHILD_NEW
0,47065010104,480.0,1003.0
1,47065010201,382.0,1102.0
2,47065010202,452.0,1240.0
3,47065010303,433.0,762.0
4,47065010304,485.0,1455.0
...,...,...,...
105,13295020301,715.0,1317.0
106,13083040300,346.0,875.0
107,13083040200,324.0,1199.0
108,13083040101,717.0,1280.0


### Get Households by Household Size

In [6]:
hh = []

for state, co in state_county_fips:
    # 1-Person Households
    hh.append(pd.DataFrame(c.acs5.get(('NAME', 'B08201_007E'),
                            {
                                'for': 'tract:*',
                                'in': 'state:{} county:{}'.format(state, co)
                            }
                ))[['state', 'county','tract', 'B08201_007E']].melt(id_vars=['state', 'county','tract']))
    
    # 2-Person Households
    hh.append(pd.DataFrame(c.acs5.get(('NAME', 'B08201_013E'),
                            {
                                'for': 'tract:*',
                                'in': 'state:{} county:{}'.format(state, co)
                            }
                ))[['state', 'county','tract', 'B08201_013E']].melt(id_vars=['state', 'county','tract']))
    
    # 3-Person Households
    hh.append(pd.DataFrame(c.acs5.get(('NAME', 'B08201_019E'),
                            {
                                'for': 'tract:*',
                                'in': 'state:{} county:{}'.format(state, co)
                            }
                ))[['state', 'county','tract', 'B08201_019E']].melt(id_vars=['state', 'county','tract']))
    
    # 4-Person Households
    hh.append(pd.DataFrame(c.acs5.get(('NAME', 'B08201_025E'),
                            {
                                'for': 'tract:*',
                                'in': 'state:{} county:{}'.format(state, co)
                            }
                ))[['state', 'county','tract', 'B08201_025E']].melt(id_vars=['state', 'county','tract']))
hh=pd.concat(hh)
hh = hh.set_index(['state', 'county', 'tract', 'variable']).unstack(-1)
hh.columns = ['hh1', 'hh2', 'hh3', 'hh4']
hh = hh.reset_index()

hh['GEOID10'] = hh.apply(lambda x: '{}{}{}'.format(x.state, x.county, x.tract), axis=1)
pd.merge(tract_data, hh[['GEOID10', 'hh1', 'hh2', 'hh3', 'hh4']], on='GEOID10')[['GEOID10', 'hh1', 'hh2', 'hh3', 'hh4']].to_clipboard(sep='\t')

### Get Total Group Quarters

In [7]:
hh = []

for state, co in state_county_fips:
    # Get the Total Count of Group Quarters
    hh.append(pd.DataFrame(c.acs5.get(('NAME', 'B26001_001E'),
                            {
                                'for': 'tract:*',
                                'in': 'state:{} county:{}'.format(state, co)
                            }
                )))
hh=pd.concat(hh)
hh['GEOID10'] = hh.apply(lambda x: '{}{}{}'.format(x.state, x.county, x.tract), axis=1)
pd.merge(tract_data, hh[['GEOID10', 'B26001_001E']], on='GEOID10')[['GEOID10', 'B26001_001E']]#.to_clipboard(sep='\t')

,GEOID10,B26001_001E
0,47065010104,64.0
1,47065010201,0.0
2,47065010202,0.0
3,47065010303,0.0
4,47065010304,0.0
...,...,...
105,13295020301,0.0
106,13083040300,1137.0
107,13083040200,79.0
108,13083040101,91.0


### Use Average  Household Size to Determine Average Size of 4+ HH
Then calculate implied population and scale Male / Female and Age Breakdown to match implied household population from household size.

In [8]:
hh = []

for state, co in state_county_fips:
    # Get the Average Household Size
    hh.append(pd.DataFrame(c.acs5.get(('NAME', 'B25010_001E'),
                            {
                                'for': 'tract:*',
                                'in': 'state:{} county:{}'.format(state, co)
                            }
                )))
    
hh=pd.concat(hh)
hh['GEOID10'] = hh.apply(lambda x: '{}{}{}'.format(x.state, x.county, x.tract), axis=1)
pd.merge(tract_data, hh[['GEOID10', 'B25010_001E']], on='GEOID10')[['GEOID10', 'B25010_001E']]#.to_clipboard(sep='\t')

,GEOID10,B25010_001E
0,47065010104,2.61
1,47065010201,2.58
2,47065010202,2.60
3,47065010303,2.59
4,47065010304,2.54
...,...,...
105,13295020301,2.65
106,13083040300,2.44
107,13083040200,2.51
108,13083040101,2.29
